# Generating QIR from Q#

First, import `qsharp` and initialize it with the desired QIR profile.

In [ ]:
import qsharp

qsharp.init(target_profile=qsharp.TargetProfile.Adaptive_RI)

Use a Q# cell to define a parametrized operation to use later from Python.

In [ ]:
%%qsharp

import Std.Arrays.Reversed;

operation TeleportRepeat(iterations : Int) : Result[] {
    use qs = Qubit[2];
    X(qs[0]);

    // Repeatedly teleport the state back and forth between the two qubits.
    mutable register = qs;
    for _ in 1..iterations {
        Teleport(register[0], register[1]);
        register = Reversed(register);
    }

    MResetEachZ(qs)
}

/// # Summary
/// Sends the state of one qubit to a target qubit by using teleportation.
///
/// Notice that after calling Teleport, the state of `message` is collapsed.
///
/// # Input
/// ## message
/// A qubit whose state we wish to send.
/// ## target
/// A qubit initially in the |0〉 state that we want to send
/// the state of message to.
operation Teleport(message : Qubit, target : Qubit) : Unit {
    // Allocate an auxiliary qubit.
    use auxiliary = Qubit();

    // Create some entanglement that we can use to send our message.
    H(auxiliary);
    CNOT(auxiliary, target);

    // Encode the message into the entangled pair.
    CNOT(message, auxiliary);
    H(message);

    // Measure the qubits to extract the classical data we need to decode
    // the message by applying the corrections on the target qubit
    // accordingly.
    if MResetZ(auxiliary) == One {
        X(target);
    }

    if MResetZ(message) == One {
        Z(target);
    }

    // Reset auxiliary qubit before releasing.
    Reset(auxiliary);
}

Now import that operation and call it from Python to verify the behavior in simulation.

In [ ]:
from qsharp.code import TeleportRepeat

# Run the code to verify behavior
iter = 4
for i in range(10):
    print(TeleportRepeat(iter))

Use the same imported operation to compile into a QIR program.

In [ ]:
# Compile to QIR
qir = qsharp.compile(TeleportRepeat, 5)
print(qir._ll_str)

Finally, verify the behavior of the generated QIR by simulating with `qirrunner`.

In [ ]:
import qirrunner
from pathlib import Path

# Save the QIR to a file
qir_path = Path("TeleportRepeat.ll")
qir_path.write_text(qir._ll_str)

# Run the QIR using qirrunner
qirrunner.run(str(qir_path), shots=10)